In [1]:
!git clone https://github.com/chankiel/Steam-Review-NLP-Pipeline.git
%cd Steam-Review-NLP-Pipeline

Cloning into 'Steam-Review-NLP-Pipeline'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 203 (delta 85), reused 153 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (203/203), 1.60 MiB | 4.10 MiB/s, done.
Resolving deltas: 100% (85/85), done.
/content/Steam-Review-NLP-Pipeline


In [2]:
import sys
sys.path.append('/content/Steam-Review-NLP-Pipeline')

In [ ]:
import pandas as pd
import torch
from src.utils.config import get_config
from src.training.models.absa_model import ABSAModel

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
absa_model = ABSAModel()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1/3 - Train Loss: 0.5516
Validation Accuracy: 0.8455


Epoch 2/3 - Train Loss: 0.3679
Validation Accuracy: 0.8716


Epoch 3/3 - Train Loss: 0.2739
Validation Accuracy: 0.8781


In [ ]:
absa_model.load_state_dict(torch.load("../../models/best/absa_model.pt"))
absa_model.eval()

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer
from src.preprocess.cleaning import clean_review_text

In [ ]:
config = get_config()
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)

In [10]:
# https://drive.google.com/file/d/1wv8Ifa9yAML5xf-4-gf5TT-2fVjNSWgM/view?usp=sharing
df = pd.read_csv('https://drive.google.com/uc?id=1wv8Ifa9yAML5xf-4-gf5TT-2fVjNSWgM')


In [ ]:
df["review_text"] = df["review_text"].apply(clean_review_text)

In [ ]:
reviews = df["review_text"].tolist()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
absa_model.to(device)
absa_model.eval()  # evaluation mode

In [ ]:
batch_size = 16  # adjust depending on your GPU memory
all_logits = []

ABSAModel(
  (base): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [ ]:
inputs = tokenizer(
    reviews,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

In [ ]:
ids = inputs['input_ids']
mask = inputs['attention_mask']

# Create a DataLoader for batching
dataset = TensorDataset(ids, mask)
dataloader = DataLoader(dataset, batch_size=batch_size)

In [ ]:
from tqdm import tqdm

with torch.no_grad():
    for batch_ids, batch_mask in tqdm(dataloader, desc="Processing batches"):
        batch_ids = batch_ids.to(device)
        batch_mask = batch_mask.to(device)

        logits = absa_model(batch_ids, batch_mask)  # (batch, num_aspects, 3)
        all_logits.append(logits.cpu())  # move to CPU to save GPU memory


In [ ]:
all_logits = torch.cat(all_logits, dim=0)
print(all_logits.shape)  # (num_reviews, num_aspects, 3)

Processing batches: 100%|██████████| 313/313 [01:08<00:00,  4.60it/s]


In [ ]:
preds = all_logits.argmax(dim=2)  # shape: (1, num_aspects)
preds = preds.squeeze(0)      # shape: (num_aspects,)

# Convert back to -1/0/1
preds = preds.cpu().numpy() - 1

torch.Size([5000, 8, 3])


In [ ]:
df_preds = pd.DataFrame(preds, columns=config.ASPECTS)

tensor([[-0.6410, -0.7024,  2.7234],
        [-0.3877, -0.5880,  1.5248],
        [ 1.4526, -1.5503,  0.2739],
        [-1.1713,  1.3889,  0.1634],
        [-1.1566,  2.2134, -1.3196],
        [-1.4331,  3.0823, -1.2330],
        [-1.9157,  2.5682, -1.0109],
        [-1.6316,  2.3431, -0.6447]])

In [ ]:
df_preds.insert(0, "app_name", df["app_name"].tolist())

In [ ]:
avg_absa_game = df_preds.groupby(
    by=["app_name"]
).agg(
    gameplay=('gameplay','mean'),
    graphics=('graphics','mean'),
    performance=('performance','mean'),
    story=('story','mean'),
    controls=('controls','mean'),
    audio=('audio','mean'),
    price=('price','mean'),
    multiplayer=('multiplayer','mean'),
).reset_index()

In [ ]:
avg_absa_game.to_csv("absa_game.csv", index=False)

In [ ]:
from google.colab import files

files.download("absa_game.csv")